#### Importing the necessary libraries

In [128]:
import azureml.core  # Core Azure ML functionality
from azureml.core import Workspace, Experiment, Environment, ScriptRunConfig  # Workspace management, environment setup, and run configuration
from azureml.core.compute import ComputeTarget, AmlCompute  # Compute target management and Azure ML compute
from azureml.core.compute_target import ComputeTargetException  # Exception handling for compute targets

In [2]:
resource_name = "Abhinav"
workspace = "ML-Workspace"
subscription_id = "4e916481-3d7f-41cf-babb-68cdb2f17989"
aml_compute_target = "medium1-10gb-compute-cluster"
experiment_name = "bank-transaction1"

#### Creating the workspace
##### We already had created the workspace so we downloaded the config.json file and used that to create the workspace

In [3]:
try:
    ws = Workspace.from_config()
    print("Workspace found")
except:
    ws = Workspace.create(name=workspace,
                         subscription_id=subscription_id,
                         resource_group=resource_name,
                         location="Southeast Asia")
    ws.write_config(path="config.json")
    

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace found


#### Creating the compute target 
#### We already had created the compute target so we did not create it again

In [4]:
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("AML Compute found")
except ComputeTargetException as e:
    print("AML Compute not found creating new AML Compute")

    provisioning_config = AmlCompute.provisioning_configuration(vm_size="Standard_A4_v2",
                                                                min_nodes=0,
                                                                max_nodes=3,
                                                                idle_seconds_before_scaledown=3000)
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True,min_node_count=None,timeout_in_minutes=15)

print("AML Compute attached")
    

AML Compute found
AML Compute attached


#### Create experiment

In [5]:
exp = Experiment(ws,experiment_name)

#### Running the experiment ( for the first time takes a while took me 23 mins)

In [11]:
env = Environment.from_conda_specification(name="azure_ml",file_path="./envfile.yml") # creating the environment from the yml file
config = ScriptRunConfig(source_directory="./",script="train.py",compute_target=aml_compute_target,environment=env) # creating the config for the experiment job
execution = exp.submit(config) # running the job in azure ML
execution.wait_for_completion(show_output=True) # this helps to see the execution of the job

RunId: bank-transaction1_1727021865_93561281
Web View: https://ml.azure.com/runs/bank-transaction1_1727021865_93561281?wsid=/subscriptions/4e916481-3d7f-41cf-babb-68cdb2f17989/resourcegroups/Abhinav/workspaces/ML-Workspace&tid=741fe87a-c689-4c98-9bcd-498bd23ee46c

Streaming azureml-logs/20_image_build_log.txt

======Starting Image Build on Serverless Compute======
The run ID for the image build on serverless compute is imgbldrun_d387f9a
Additional logs for the run: https://ml.azure.com/experiments/id/prepare_image/runs/imgbldrun_d387f9a?wsid=/subscriptions/4e916481-3d7f-41cf-babb-68cdb2f17989/resourcegroups/Abhinav/workspaces/ML-Workspace&tid=741fe87a-c689-4c98-9bcd-498bd23ee46c
2024-09-22T16:22:01: Logging into Docker registry: 28ee59a0b42142d787174d32584776ef.azurecr.io
2024-09-22T16:22:01: WARNING! Using --password via the CLI is insecure. Use --password-stdin.
2024-09-22T16:22:01: WARNING! Your password will be stored unencrypted in /root/.docker/config.json.
2024-09-22T16:22:01: C

{'runId': 'bank-transaction1_1727021865_93561281',
 'target': 'medium1-10gb-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-09-22T16:37:24.40554Z',
 'endTimeUtc': '2024-09-22T16:41:06.152456Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  '_azureml.ClusterName': 'medium1-10gb-compute-cluster',
  'ContentSnapshotId': 'b850a5b4-8888-4a10-a60d-0488e806e3d3',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'medium1-10gb-compute-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  

In [13]:
config=ScriptRunConfig(source_directory="./",script="train.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7]) # this time we are passing the arguments to the config i.e. hyperparameters
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

In [14]:
config=ScriptRunConfig(source_directory="./",script="train_metric_log.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7]) # we use train_metric_log.py and store the metrics in azure ML
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

RunId: bank-transaction1_1727025476_facf3f23
Web View: https://ml.azure.com/runs/bank-transaction1_1727025476_facf3f23?wsid=/subscriptions/4e916481-3d7f-41cf-babb-68cdb2f17989/resourcegroups/Abhinav/workspaces/ML-Workspace&tid=741fe87a-c689-4c98-9bcd-498bd23ee46c

Streaming user_logs/std_log.txt

Index(['Unnamed: 0', 'AGE', 'SEX', 'BMI', 'BP', 'S1', 'S2', 'S3', 'S4', 'S5',
       'S6', 'Y'],
      dtype='object')
Accuracy:  0.7191011235955056
Classification Report:
               precision    recall  f1-score   support

           1       0.69      0.85      0.77        48
           2       0.77      0.56      0.65        41

    accuracy                           0.72        89
   macro avg       0.73      0.71      0.71        89
weighted avg       0.73      0.72      0.71        89

ROC AUC Score:  0.7075711382113821
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 5.359530687332153 seconds

Execution Summary
RunId: bank-transact

{'runId': 'bank-transaction1_1727025476_facf3f23',
 'target': 'medium1-10gb-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-09-22T17:18:36.905409Z',
 'endTimeUtc': '2024-09-22T17:19:03.145167Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  '_azureml.ClusterName': 'medium1-10gb-compute-cluster',
  'ContentSnapshotId': 'a4af988d-9d72-4a6b-bd74-0d2374a2e20e',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_metric_log.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--min_samples_leaf', '5', '--min_samples_split', '7'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'medium1-10gb-compute-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000

In [33]:
config=ScriptRunConfig(source_directory="./",script="train_metric_log.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7]) # we use train_metric_log.py and store the metrics in azure ML again running by saving the model in the outputs folder
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

RunId: bank-transaction1_1727027239_c36e42c4
Web View: https://ml.azure.com/runs/bank-transaction1_1727027239_c36e42c4?wsid=/subscriptions/4e916481-3d7f-41cf-babb-68cdb2f17989/resourcegroups/Abhinav/workspaces/ML-Workspace&tid=741fe87a-c689-4c98-9bcd-498bd23ee46c

Streaming user_logs/std_log.txt

Index(['Unnamed: 0', 'AGE', 'SEX', 'BMI', 'BP', 'S1', 'S2', 'S3', 'S4', 'S5',
       'S6', 'Y'],
      dtype='object')
Accuracy:  0.7191011235955056
Classification Report:
               precision    recall  f1-score   support

           1       0.69      0.85      0.77        48
           2       0.77      0.56      0.65        41

    accuracy                           0.72        89
   macro avg       0.73      0.71      0.71        89
weighted avg       0.73      0.72      0.71        89

ROC AUC Score:  0.7075711382113821
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 5.355510711669922 seconds

Execution Summary
RunId: bank-transact

{'runId': 'bank-transaction1_1727027239_c36e42c4',
 'target': 'medium1-10gb-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-09-22T17:47:52.714892Z',
 'endTimeUtc': '2024-09-22T17:48:18.895016Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  '_azureml.ClusterName': 'medium1-10gb-compute-cluster',
  'ContentSnapshotId': 'f7c40548-48de-4f91-9c03-fd22cc8c3a50',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_metric_log.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--min_samples_leaf', '5', '--min_samples_split', '7'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'medium1-10gb-compute-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000

In [35]:
config=ScriptRunConfig(source_directory="./",script="train_metric_log.py",compute_target=aml_compute_target,environment=env,
                            arguments=['--min_samples_leaf',5,
                                       '--min_samples_split',7]) # we use train_metric_log.py and store the metrics in azure ML again running by saving the model in the outputs folder
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)

RunId: bank-transaction1_1727027602_939fe449
Web View: https://ml.azure.com/runs/bank-transaction1_1727027602_939fe449?wsid=/subscriptions/4e916481-3d7f-41cf-babb-68cdb2f17989/resourcegroups/Abhinav/workspaces/ML-Workspace&tid=741fe87a-c689-4c98-9bcd-498bd23ee46c

Streaming user_logs/std_log.txt

Index(['Unnamed: 0', 'AGE', 'SEX', 'BMI', 'BP', 'S1', 'S2', 'S3', 'S4', 'S5',
       'S6', 'Y'],
      dtype='object')
Accuracy:  0.7191011235955056
Classification Report:
               precision    recall  f1-score   support

           1       0.69      0.85      0.77        48
           2       0.77      0.56      0.65        41

    accuracy                           0.72        89
   macro avg       0.73      0.71      0.71        89
weighted avg       0.73      0.72      0.71        89

ROC AUC Score:  0.7075711382113821
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 5.3548903465271 seconds

Execution Summary
RunId: bank-transactio

{'runId': 'bank-transaction1_1727027602_939fe449',
 'target': 'medium1-10gb-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2024-09-22T17:53:55.2871Z',
 'endTimeUtc': '2024-09-22T17:54:21.910653Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  '_azureml.ClusterName': 'medium1-10gb-compute-cluster',
  'ContentSnapshotId': 'c0795016-9ab8-4155-a5e1-45702bdceb8d',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train_metric_log.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--min_samples_leaf', '5', '--min_samples_split', '7'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'medium1-10gb-compute-cluster',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,


#### Register Model to Azure ML Models

In [36]:
# I want to see all the run ids of the experiment along with their metrics
for run in exp.get_runs():
    print(run.id, run.get_metrics())

bank-transaction1_1727027602_939fe449 {'Parameter': "{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}", 'Accuracy': 0.7191011235955056}
bank-transaction1_1727027239_c36e42c4 {'Accuracy': 0.7191011235955056, 'Parameter': "{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}"}


In [37]:
from azureml.core import Run

# Get the run for the specific run id
run = Run.get(workspace=ws, run_id="bank-transaction1_1727027602_939fe449")


In [60]:
# register your model from your run
model = run.register_model(model_name='bankdatamodel', model_path='outputs/bankdatamodel.pkl')

#### Model is registered now lets deploy the model

In [157]:
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

inferconfig = InferenceConfig(runtime='python',source_directory='./',entry_script='score.py',conda_file="./envfile.yml")

In [158]:
from azureml.core.model import Model
model = Model(workspace=ws, name='bankdatamodel')

In [159]:
model

Model(workspace=Workspace.create(name='ML-Workspace', subscription_id='4e916481-3d7f-41cf-babb-68cdb2f17989', resource_group='Abhinav'), name=bankdatamodel, id=bankdatamodel:2, version=2, tags={}, properties={})

In [160]:
myenv = Environment(name="demo-env")
conda_packages = ['numpy']
pip_packages = ['azureml-sdk','azureml-defaults','scikit-learn','pandas']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages,python_version='3.8.10')
myenv.python.conda_dependencies = mycondaenv
myenv.register(workspace = ws)

inference_config = InferenceConfig(entry_script="score.py", source_directory='./',environment=myenv)

In [161]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,memory_gb=1)

In [162]:
# Deploying the model
service = Model.deploy(ws,"demomodel", #This is endpoint name
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aciconfig,
                           #deployment_target=predenv,
                           overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

C:\Users\manea\AppData\Local\Temp\ipykernel_10804\1534230109.py:2: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws,"demomodel", #This is endpoint name


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-09-23 22:44:41+05:30 Creating Container Registry if not exists.
2024-09-23 22:44:44+05:30 Use the existing image.
2024-09-23 22:44:44+05:30 Generating deployment configuration.
2024-09-23 22:44:46+05:30 Submitting deployment to compute.
2024-09-23 22:44:51+05:30 Checking the status of deployment demomodel..
2024-09-23 22:47:42+05:30 Checking the status of inference endpoint demomodel.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://a837dca4-4600-481c-b1a1-7ad46a4d2a68.southeastasia.azurecontainer.io/score


#### prepare data for testing

In [163]:
# Prepare your test data
import pandas as pd
import json
data = pd.read_csv('bankdata.csv')
data = data.drop("SEX",axis=1)
xt = data.values.tolist()
test = json.dumps({"data":xt})

In [164]:
test

'{"data": [[0.0, 59.0, 32.1, 101.0, 157.0, 93.2, 38.0, 4.0, 4.8598, 87.0, 151.0], [1.0, 48.0, 21.6, 87.0, 183.0, 103.2, 70.0, 3.0, 3.8918, 69.0, 75.0], [2.0, 72.0, 30.5, 93.0, 156.0, 93.6, 41.0, 4.0, 4.6728, 85.0, 141.0], [3.0, 24.0, 25.3, 84.0, 198.0, 131.4, 40.0, 5.0, 4.8903, 89.0, 206.0], [4.0, 50.0, 23.0, 101.0, 192.0, 125.4, 52.0, 4.0, 4.2905, 80.0, 135.0], [5.0, 23.0, 22.6, 89.0, 139.0, 64.8, 61.0, 2.0, 4.1897, 68.0, 97.0], [6.0, 36.0, 22.0, 90.0, 160.0, 99.6, 50.0, 3.0, 3.9512, 82.0, 138.0], [7.0, 66.0, 26.2, 114.0, 255.0, 185.0, 56.0, 4.55, 4.2485, 92.0, 63.0], [8.0, 60.0, 32.1, 83.0, 179.0, 119.4, 42.0, 4.0, 4.4773, 94.0, 110.0], [9.0, 29.0, 30.0, 85.0, 180.0, 93.4, 43.0, 4.0, 5.3845, 88.0, 310.0], [10.0, 22.0, 18.6, 97.0, 114.0, 57.6, 46.0, 2.0, 3.9512, 83.0, 101.0], [11.0, 56.0, 28.0, 85.0, 184.0, 144.8, 32.0, 6.0, 3.5835, 77.0, 69.0], [12.0, 53.0, 23.7, 92.0, 186.0, 109.2, 62.0, 3.0, 4.3041, 81.0, 179.0], [13.0, 50.0, 26.2, 97.0, 186.0, 105.4, 49.0, 4.0, 5.0626, 88.0, 185.0

In [166]:
service.scoring_uri

'http://a837dca4-4600-481c-b1a1-7ad46a4d2a68.southeastasia.azurecontainer.io/score'

In [168]:
import requests
import json

# Replace with your scoring URI
scoring_uri = "http://a837dca4-4600-481c-b1a1-7ad46a4d2a68.southeastasia.azurecontainer.io/score"

# Example input data
input_data = {
    "data": [
        [12344.0, 59.0, 32.1, 101.0, 157.0, 93.2, 38.0, 4.0, 4.8598, 87.0, 151.0],
    ]
}

# Convert input data to JSON string
input_json = json.dumps(input_data)

# Set the content type
headers = {"Content-Type": "application/json"}

# Send POST request
response = requests.post(scoring_uri, data=input_json, headers=headers)

# Print the response
print(f"Response: {response.json()}")


Response: {"result": [2]}
